In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [2]:
data_df = pd.read_csv('train_val_data.csv', index_col=False)
data_df.head()

,ChipsetBrand,CardDescription,NumberOfUSB2.0Ports,NumberOfUSB3.0Ports,BrandName,OperatingSystem,ProcessorBrand,ProcessorCount,HardDriveInterface,HardwarePlatform,...,HardDrive_measure_pcs,ItemWeight_pcs,ItemDimensions_L_pcs,ItemDimensions_W_pcs,ItemDimensions_H_pcs,AverageBatteryLife_pcs,Batteries_pcs,GraphicsCardRamSize_pcs,FlashMemorySize_pcs,Processor_ghz_pcs
0,intel,Integrated,2.0,NaN,Dell,Windows 10,Intel,1.0,Serial ATA,PC,...,TB,5.20,14.96,10.25,0.94,5.5,1 Lithium ion batteries required,NaN,NaN,3.1
1,nvidia,dedicated,2.0,NaN,Lenovo,NaN,NaN,1.0,Serial ATA,NaN,...,NaN,5.51,14.90,9.90,1.20,NaN,NaN,0.0,0.01,NaN
2,nvidia,dedicated,NaN,NaN,Dell,Windows 10,Intel,4.0,PCIE x 4,PC,...,GB,4.23,14.40,10.00,0.85,NaN,1 Lithium ion batteries required,NaN,512.00,2.4
3,NaN,integrated,6.0,NaN,Lenovo,Windows 10,Intel,2.0,Serial ATA,PC,...,TB,14.55,15.00,3.90,13.20,NaN,NaN,NaN,NaN,3.3
4,intel,NaN,NaN,NaN,Dell,NaN,NaN,2.0,NaN,NaN,...,NaN,6.60,21.70,13.80,3.30,NaN,1 Lithium ion batteries required,NaN,512.00,NaN


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5107 entries, 0 to 5106
Data columns (total 27 columns):
ChipsetBrand                      3493 non-null object
CardDescription                   3849 non-null object
NumberOfUSB2.0Ports               1518 non-null float64
NumberOfUSB3.0Ports               1170 non-null float64
BrandName                         5087 non-null object
OperatingSystem                   4587 non-null object
ProcessorBrand                    4450 non-null object
ProcessorCount                    5000 non-null float64
HardDriveInterface                3827 non-null object
HardwarePlatform                  3907 non-null object
Price_pcs                         5107 non-null float64
ScreenSize_pcs                    4725 non-null float64
MaxScreenResolution_width_pcs     3718 non-null float64
MaxScreenResolution_height_pcs    3718 non-null float64
RAM_capacity_pcs                  4974 non-null float64
RAM_type_pcs                      3698 non-null object
HardD

# Split Data

In [4]:
y_sr = data_df['Price_pcs']
X_df = data_df.drop('Price_pcs', axis=1)

In [5]:
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.15, random_state=0)

In [6]:
train_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4596 entries, 3775 to 2732
Data columns (total 26 columns):
ChipsetBrand                      3144 non-null object
CardDescription                   3463 non-null object
NumberOfUSB2.0Ports               1365 non-null float64
NumberOfUSB3.0Ports               1046 non-null float64
BrandName                         4578 non-null object
OperatingSystem                   4126 non-null object
ProcessorBrand                    4008 non-null object
ProcessorCount                    4500 non-null float64
HardDriveInterface                3460 non-null object
HardwarePlatform                  3531 non-null object
ScreenSize_pcs                    4247 non-null float64
MaxScreenResolution_width_pcs     3339 non-null float64
MaxScreenResolution_height_pcs    3339 non-null float64
RAM_capacity_pcs                  4480 non-null float64
RAM_type_pcs                      3345 non-null object
HardDrive_capacity_pcs            3852 non-null float64
Ha

# Transform data

In [7]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        transformed_X_df = X_df.copy()
        transformed_X_df['GraphicsCardRamSize_pcs'] = transformed_X_df['GraphicsCardRamSize_pcs'].fillna(value=0.0, inplace=False)
        transformed_X_df = transformed_X_df.drop(['NumberOfUSB2.0Ports', 'NumberOfUSB3.0Ports', 'ScreenSize_pcs', 'FlashMemorySize_pcs'], axis=1)
        return transformed_X_df

In [8]:
class FillNA(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.fillna_values = {}
    def fit(self, X_df, y=None):
        self.fillna_values['ProcessorCount'] = X_df['ProcessorCount'].mean()
#         self.fillna_values['ScreenSize_pcs'] = X_df['ScreenSize_pcs'].mean()
        self.fillna_values['MaxScreenResolution_width_pcs'] = X_df['MaxScreenResolution_width_pcs'].mean()
        self.fillna_values['MaxScreenResolution_height_pcs'] = X_df['MaxScreenResolution_height_pcs'].mean()
        self.fillna_values['HardDrive_capacity_pcs'] = X_df['HardDrive_capacity_pcs'].mean()
        self.fillna_values['ItemDimensions_L_pcs'] = X_df['ItemDimensions_L_pcs'].mean()
        self.fillna_values['ItemDimensions_W_pcs'] = X_df['ItemDimensions_W_pcs'].mean()
        self.fillna_values['ItemDimensions_H_pcs'] = X_df['ItemDimensions_H_pcs'].mean()
        self.fillna_values['AverageBatteryLife_pcs'] = X_df['AverageBatteryLife_pcs'].mean()
        self.fillna_values['GraphicsCardRamSize_pcs'] = X_df['GraphicsCardRamSize_pcs'].mean()
#         self.fillna_values['FlashMemorySize_pcs'] = X_df['FlashMemorySize_pcs'].mean()
        self.fillna_values['Processor_ghz_pcs'] = X_df['Processor_ghz_pcs'].mean()
        self.fillna_values['RAM_capacity_pcs'] = X_df['RAM_capacity_pcs'].mean()
        self.fillna_values['ItemWeight_pcs'] = X_df['ItemWeight_pcs'].mean()
        
        self.fillna_values['ChipsetBrand'] = X_df['ChipsetBrand'].mode()[0]
        self.fillna_values['CardDescription'] = X_df['CardDescription'].mode()[0]
        self.fillna_values['BrandName'] = X_df['BrandName'].mode()[0]
        self.fillna_values['OperatingSystem'] = X_df['OperatingSystem'].mode()[0]
        self.fillna_values['ProcessorBrand'] = X_df['ProcessorBrand'].mode()[0]
        self.fillna_values['HardDriveInterface'] = X_df['HardDriveInterface'].mode()[0]
        self.fillna_values['HardwarePlatform'] = X_df['HardwarePlatform'].mode()[0]
        self.fillna_values['RAM_type_pcs'] = X_df['RAM_type_pcs'].mode()[0]
        self.fillna_values['HardDrive_measure_pcs'] = X_df['HardDrive_measure_pcs'].mode()[0]
        self.fillna_values['Batteries_pcs'] = X_df['Batteries_pcs'].mode()[0]
            
        return self
    
    def transform(self, X_df, y=None):
        transformed_X_df = X_df.copy()
        
        transformed_X_df['ProcessorCount'].fillna(value=self.fillna_values['ProcessorCount'], inplace=True)
#         transformed_X_df['ScreenSize_pcs'].fillna(value=self.fillna_values['ScreenSize_pcs'], inplace=True)
        transformed_X_df['MaxScreenResolution_width_pcs'].fillna(value=self.fillna_values['MaxScreenResolution_width_pcs'], inplace=True)
        transformed_X_df['MaxScreenResolution_height_pcs'].fillna(value=self.fillna_values['MaxScreenResolution_height_pcs'], inplace=True)
        transformed_X_df['HardDrive_capacity_pcs'].fillna(value=self.fillna_values['HardDrive_capacity_pcs'], inplace=True)
        transformed_X_df['ItemDimensions_L_pcs'].fillna(value=self.fillna_values['ItemDimensions_L_pcs'], inplace=True)
        transformed_X_df['ItemDimensions_W_pcs'].fillna(value=self.fillna_values['ItemDimensions_W_pcs'], inplace=True)
        transformed_X_df['ItemDimensions_H_pcs'].fillna(value=self.fillna_values['ItemDimensions_H_pcs'], inplace=True)
        transformed_X_df['AverageBatteryLife_pcs'].fillna(value=self.fillna_values['AverageBatteryLife_pcs'], inplace=True)
        transformed_X_df['GraphicsCardRamSize_pcs'].fillna(value=self.fillna_values['GraphicsCardRamSize_pcs'], inplace=True)
#         transformed_X_df['FlashMemorySize_pcs'].fillna(value=self.fillna_values['FlashMemorySize_pcs'], inplace=True)
        transformed_X_df['Processor_ghz_pcs'].fillna(value=self.fillna_values['Processor_ghz_pcs'], inplace=True)
        transformed_X_df['RAM_capacity_pcs'].fillna(value=self.fillna_values['RAM_capacity_pcs'], inplace=True)
        transformed_X_df['ItemWeight_pcs'].fillna(value=self.fillna_values['ItemWeight_pcs'], inplace=True)
        
        transformed_X_df['ChipsetBrand'].fillna(value=self.fillna_values['ChipsetBrand'], inplace=True)
        transformed_X_df['CardDescription'].fillna(value=self.fillna_values['CardDescription'], inplace=True)
        transformed_X_df['BrandName'].fillna(value=self.fillna_values['BrandName'], inplace=True)
        transformed_X_df['OperatingSystem'].fillna(value=self.fillna_values['OperatingSystem'], inplace=True)
        transformed_X_df['ProcessorBrand'].fillna(value=self.fillna_values['ProcessorBrand'], inplace=True)
        transformed_X_df['HardDriveInterface'].fillna(value=self.fillna_values['HardDriveInterface'], inplace=True)
        transformed_X_df['HardwarePlatform'].fillna(value=self.fillna_values['HardwarePlatform'], inplace=True)
        transformed_X_df['RAM_type_pcs'].fillna(value=self.fillna_values['RAM_type_pcs'], inplace=True)
        transformed_X_df['HardDrive_measure_pcs'].fillna(value=self.fillna_values['HardDrive_measure_pcs'], inplace=True)
        transformed_X_df['Batteries_pcs'].fillna(value=self.fillna_values['Batteries_pcs'], inplace=True)
        
        return transformed_X_df

In [9]:
# col_adderdropper = ColAdderDropper()
# train_X_df = col_adderdropper.fit_transform(train_X_df)
# val_X_df = col_adderdropper.fit_transform(val_X_df)

In [10]:
# fillNa = FillNA()
# fillNa.fit(train_X_df)
# train_X_df = fillNa.transform(train_X_df)
# train_X_df.info()

In [11]:
# val_X_df = fillNa.transform(val_X_df)
# val_X_df.info()

# Full Pipeline - Train

In [12]:
from sklearn import linear_model, svm

In [21]:
numeric_cols = ['ProcessorCount', 'ScreenSize_pcs', 'MaxScreenResolution_width_pcs', 'MaxScreenResolution_height_pcs', 'RAM_capacity_pcs', 
                'HardDrive_capacity_pcs', 'ItemWeight_pcs', 'ItemDimensions_L_pcs', 'ItemDimensions_W_pcs', 'ItemDimensions_H_pcs', 
               'AverageBatteryLife_pcs', 'GraphicsCardRamSize_pcs', 'FlashMemorySize_pcs', 'Processor_ghz_pcs']
category_cols = ['ChipsetBrand', 'CardDescription', 'BrandName', 'OperatingSystem', 'ProcessorBrand', 'HardDriveInterface', 'HardwarePlatform'
               , 'RAM_type_pcs', 'HardDrive_measure_pcs', 'Batteries_pcs']

numeric_imputer = Pipeline([('imputer', SimpleImputer(strategy='mean'))])
category_imputer = Pipeline([('one_hot_vector', OneHotEncoder(handle_unknown='ignore'))])

transform_cols = ColumnTransformer([('category_imputer', category_imputer, category_cols)])
reprocess_pipeline = Pipeline([('coladderdropper', ColAdderDropper()), ('fill_na', FillNA()), ('transform_cols', transform_cols), ('std_scaler', StandardScaler(with_mean=False))])

In [29]:
svm_ = svm.SVC(verbose=False, random_state=0, max_iter=100)
full_pipeline = Pipeline([('reprocess_data', reprocess_pipeline), ('svm', svm_)])

max_iter = [50000]
for iter_ in max_iter:
    full_pipeline.set_params(svm__max_iter=iter_)
    full_pipeline.fit(train_X_df, train_y_sr)
    print('MaxIter: %f - Train acc: %f - Val acc: %f' %(iter_, full_pipeline.score(train_X_df, train_y_sr), full_pipeline.score(val_X_df, val_y_sr)))

MaxIter: 50000.000000 - Train acc: 0.615318 - Val acc: 0.532290


# MLPClassifier

In [27]:
mlp_classifier = MLPClassifier((40,), 'tanh', 'lbfgs', max_iter=200, random_state=0, verbose=False)
full_pipeline = Pipeline([('reprocess_data', reprocess_pipeline), ('mlp_classifier', mlp_classifier)])

alphas = [0.1, 1, 10]
max_iter = [200, 500, 1000]
for alpha in alphas:
    for iter_ in max_iter:
        full_pipeline.set_params(mlp_classifier__alpha=alpha, mlp_classifier__max_iter=iter_)
        full_pipeline.fit(train_X_df, train_y_sr)
        print('Alpha: %f - MaxIter: %f - Train acc: %f - Val acc: %f' %(alpha, iter_, full_pipeline.score(train_X_df, train_y_sr), full_pipeline.score(val_X_df, val_y_sr)))

c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Alpha: 0.100000 - MaxIter: 1000.000000 - Train acc: 0.842689 - Val acc: 0.502935


c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Alpha: 0.100000 - MaxIter: 2000.000000 - Train acc: 0.842907 - Val acc: 0.508806


c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

Alpha: 1.000000 - MaxIter: 3000.000000 - Train acc: 0.842907 - Val acc: 0.499022


c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Alpha: 10.000000 - MaxIter: 1000.000000 - Train acc: 0.833551 - Val acc: 0.526419


c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Alpha: 10.000000 - MaxIter: 2000.000000 - Train acc: 0.832245 - Val acc: 0.538160
Alpha: 10.000000 - MaxIter: 3000.000000 - Train acc: 0.832681 - Val acc: 0.538160


# Logistic regression

In [23]:
logistic_regression = LogisticRegression(solver='lbfgs', max_iter=500, verbose=0, n_jobs=4, random_state=0)
full_pipeline = Pipeline([('reprocess_data', reprocess_pipeline), ('logistic_regression', logistic_regression)])

max_iter = [100, 200, 300, 500, 1000]
for iter_ in max_iter:
    full_pipeline.set_params(logistic_regression__max_iter=iter_)
    full_pipeline.fit(train_X_df, train_y_sr)
    print('MaxIter: %f - Train acc: %f - Val acc: %f' %(iter_, full_pipeline.score(train_X_df, train_y_sr), full_pipeline.score(val_X_df, val_y_sr)))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    1.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


MaxIter: 100.000000 - Train acc: 0.632942 - Val acc: 0.547945


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


MaxIter: 200.000000 - Train acc: 0.633377 - Val acc: 0.547945


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


MaxIter: 300.000000 - Train acc: 0.633594 - Val acc: 0.547945


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


MaxIter: 500.000000 - Train acc: 0.633594 - Val acc: 0.544031
MaxIter: 1000.000000 - Train acc: 0.633594 - Val acc: 0.542074


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    1.0s finished


# SVM

In [24]:
svm_ = svm.SVC(verbose=False, random_state=0, max_iter=100)
full_pipeline = Pipeline([('reprocess_data', reprocess_pipeline), ('svm', svm_)])

max_iter = [300, 500, 1000, 2000, 4000]
for iter_ in max_iter:
    full_pipeline.set_params(svm__max_iter=iter_)
    full_pipeline.fit(train_X_df, train_y_sr)
    print('MaxIter: %f - Train acc: %f - Val acc: %f' %(iter_, full_pipeline.score(train_X_df, train_y_sr), full_pipeline.score(val_X_df, val_y_sr)))

[LibSVM]

c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


MaxIter: 300.000000 - Train acc: 0.374021 - Val acc: 0.375734
[LibSVM]

c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


MaxIter: 500.000000 - Train acc: 0.367058 - Val acc: 0.365949
[LibSVM]

c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


MaxIter: 1000.000000 - Train acc: 0.562228 - Val acc: 0.510763
[LibSVM]

c:\users\huynhnhut\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


MaxIter: 2000.000000 - Train acc: 0.614883 - Val acc: 0.530333
[LibSVM]MaxIter: 4000.000000 - Train acc: 0.615318 - Val acc: 0.532290


# Randomforest

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
random_forest = RandomForestClassifier(n_estimators=200, verbose=0, n_jobs=4, random_state=0, oob_score=True)
full_pipeline = Pipeline([('reprocess_data', reprocess_pipeline), ('random_forest', random_forest)])

n_estimators = [100, 200, 300]
for n_es_ in n_estimators:
    full_pipeline.set_params(random_forest__n_estimators=n_es_)
    full_pipeline.fit(train_X_df, train_y_sr)
    print('n_estimator: %d - Train acc: %f - Val acc: %f' %(n_es_, full_pipeline.score(train_X_df, train_y_sr), full_pipeline.score(val_X_df, val_y_sr)))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


n_estimator: 100 - Train acc: 0.842907 - Val acc: 0.536204


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    2.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.0s finished


n_estimator: 200 - Train acc: 0.842907 - Val acc: 0.528376


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    3.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


n_estimator: 300 - Train acc: 0.842907 - Val acc: 0.538160


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished
